# DiffractionOnly_v1 Overlay Test

Este notebook carga el overlay `DiffractionOnly_v1` y compara la salida del kernel con una implementación en software.
Asegúrate de colocar `DiffractionOnly_v1.bit` y `DiffractionOnly_v1.hwh` en esta carpeta antes de ejecutar.

In [ ]:
from pynq import Overlay, allocate
import numpy as np
import time

In [ ]:
# Cargar overlay
ol = Overlay('DiffractionOnly_v1.bit')

# Instancia del IP - cambia el nombre si es diferente
ip = ol.diffraction_only_v1_0

In [ ]:
# Implementación en software como referencia
def diffraction_sw(signal):
    kernel = np.array([0.25, 0.5, 0.25], dtype=np.float32)
    return np.convolve(signal, kernel, mode='same')

In [ ]:
# Generar datos de prueba
N = 1024
in_data = np.random.rand(N).astype(np.float32)

# Software
t0 = time.time()
sw_out = diffraction_sw(in_data)
sw_time = time.time() - t0
print(f'Software time: {sw_time*1e3:.2f} ms')

In [ ]:
# Hardware
in_buffer = allocate(shape=(N,), dtype=np.float32)
out_buffer = allocate(shape=(N,), dtype=np.float32)
np.copyto(in_buffer, in_data)

t0 = time.time()
# Escribe las direcciones de memoria; ajusta offsets si es necesario
ip.write(0x10, in_buffer.physical_address)
ip.write(0x18, out_buffer.physical_address)
ip.write(0x00, 1)  # ap_start
while (ip.read(0x00) & 0x2) == 0:
    pass
hw_time = time.time() - t0
hw_out = np.array(out_buffer)
print(f'Hardware time: {hw_time*1e3:.2f} ms')

In [ ]:
# Validación
diff = np.abs(sw_out - hw_out)
print('Max error:', diff.max())
print('HW faster by {:.2f}x'.format(sw_time / hw_time))